<div style="display: flex; align-items: left;">
    <a href="https://sites.google.com/corp/google.com/genai-solutions/home?authuser=0">
        <img src="https://storage.googleapis.com/miscfilespublic/Linkedin%20Banner%20%E2%80%93%202.png" style="margin-right">
    </a>
</div>

In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


<h1 align="center">Open Data QnA - Chat with your SQL Database</h1> 


The pipeline run covers the following steps: 

> 1. Take user question and generate sql in the dialect corresponding to data source

> 2. Execute the sql query and retreive the data

> 3. Generate natural language respose and charts to display

> 4. Clean Up resources



### 📒 Using this interactive notebook

If you have not used this IDE with jupyter notebooks it will ask for installing Python + Jupyter extensions. Please go ahead install them

Click the **run** icons ▶️  of each cell within this notebook.

> 💡 Alternatively, you can run the currently selected cell with `Ctrl + Enter` (or `⌘ + Enter` on a Mac).

> ⚠️ **To avoid any errors**, wait for each section to finish in their order before clicking the next “run” icon.

This sample must be connected to a **Google Cloud project**, but nothing else is needed other than your Google Cloud project.

You can use an existing project. Alternatively, you can create a new Cloud project [with cloud credits for free.](https://cloud.google.com/free/docs/gcp-free-tier)

# 🚧 **0. Pre-requisites**

Make sure that you have completed the intial setup process using [1_Setup_OpenDataQnA.ipynb](1_Setup_OpenDataQnA.ipynb). If the 1_Setup_OpenDataQnA notebook has been run successfully, the following are set up:
* GCP project and all the required IAM permissions

* Environment to run the solution

* Data source and Vector store for the solution


## ⚙️ **1. Retreive Configuration Parameters**
The notebook will load all the configuration parameters from the `config.ini` file in the root directory. 
Most of these parameters were set in the initial notebook `1_Setup_OpenDataQnA.ipynb` and save to the 'config.ini file.
Use the below cells to retrieve these values and specify additional ones required for this notebook. 

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

Read your `PROJECT_ID` from the config.ini file, or set it manually below: 

In [ ]:
from utilities import PROJECT_ID

#PROJECT_ID = ''

## 🔐 **2. Authenticate and Connect to Google Cloud Project**
Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

You can do this within Google Colab or using the Application Default Credentials in the Google Cloud CLI.

In [ ]:
"""Colab Auth""" 
# from google.colab import auth
# auth.authenticate_user()


"""Google CLI Auth"""
# !gcloud auth application-default login


import google.auth
credentials, project_id = google.auth.default()

# Configure gcloud.
!gcloud config set project {PROJECT_ID}
print(f'Project has been set to {PROJECT_ID}')
# !gcloud auth application-default set-quota-project {PROJECT_ID}

import os
os.environ['GOOGLE_CLOUD_QUOTA_PROJECT']=PROJECT_ID
os.environ['GOOGLE_CLOUD_PROJECT']=PROJECT_ID

## ▶️ **3. Run the Open Data QnA Pipeline**

###  ❓ **Ask your Natural Language Question**

In [ ]:
# Suggested question for 'fda_food' dataset: "What are the top 5 cities with highest recalls?"
# Suggested question for 'google_dei' dataset: "How many asian men were part of the leadership workforce in 2021?"

# user_question = input(prompt_for_question) #Uncomment if you want to ask question yourself
user_question = 'How many movies have a rating higher than four?' # Or Enter Question here
user_question = "How many movies have review ratings above 5?"
#user_question = 'What are top 5 cities with highest recalls?'
print("User Question:- "+user_question)

### 🏃 **Run the Pipeline**

The next cell executes the pipeline for answering natural language questions over structured data.

The pipeline performs the following steps:

1. **Agent Loading:** Initializes various agents for embedding questions, building SQL queries, validating SQL, debugging SQL, and generating responses.
2. **Data Source and Vector Store Configuration:** Sets the data source (BigQuery or PostgreSQL) and vector store (BigQuery or PostgreSQL) based on provided parameters.
3. **Exact Match Search:** Attempts to find an exact match for the user's question in a knowledge graph cache (if enabled). If found, the cached SQL query is used.
4. **Similar Match and Schema Retrieval:** If no exact match is found, retrieves similar questions and associated SQL queries from the knowledge graph (if enabled). Also retrieves relevant table and column schemas based on similarity to the question.
5. **SQL Generation and Debugging:** Builds an initial SQL query using the retrieved information. If enabled, the debugger iteratively refines the query with potential validation and error feedback.
6. **SQL Execution and Response Generation:** Executes the final SQL query (if enabled) against the data source and retrieves results. A response agent then generates a natural language answer based on the results.
7. **Auditing:** Records the entire pipeline process, including generated SQL, responses, and potential errors, for later analysis.

Args:

* **user_question (str):** The natural language question to answer.

* **user_database (str):** Based on table schema name the value for user_database would be pg_schema or bq_dataset_name that will be used to decode the source type later

* **RUN_DEBUGGER (bool, optional):** Whether to run the SQL debugger. Defaults to True.

* **EXECUTE_FINAL_SQL (bool, optional):** Whether to execute the final SQL query. Defaults to True.

* **DEBUGGING_ROUNDS (int, optional):** The number of debugging rounds. Defaults to 2.

* **LLM_VALIDATION (bool, optional):** Whether to use LLM for SQL validation during debugging. Defaults to True.

* **Embedder_model (str, optional):** The name of the embedding model. Defaults to 'vertex'.

* **SQLBuilder_model (str, optional):** The name of the SQL building model. Defaults to 'gemini-1.0-pro'.

* **SQLChecker_model (str, optional):** The name of the SQL validation model. Defaults to 'gemini-1.0-pro'.

* **SQLDebugger_model (str, optional):** The name of the SQL debugging model. Defaults to 'gemini-1.0-pro'.

* **Responder_model (str, optional):** The name of the response generation model. Defaults to 'gemini-1.0-pro'.

* **num_table_matches (int, optional):** The number of similar tables to retrieve. Defaults to 5.

* **num_column_matches (int, optional):** The number of similar columns to retrieve. Defaults to 10.

* **table_similarity_threshold (float, optional):** The similarity threshold for tables. Defaults to 0.3.

* **column_similarity_threshold (float, optional):** The similarity threshold for columns. Defaults to 0.3.

* **example_similarity_threshold (float, optional):** The similarity threshold for example questions. Defaults to 0.3.

* **num_sql_matches (int, optional):** The number of similar SQL queries to retrieve. Defaults to 3.


Returns:
- **final_sql (str):** The final generated SQL query.
- **response (pandas.DataFrame or str):** The result of executing the SQL query or an error message.
- **_resp (str):** The final response generated by the response agent.

In [ ]:
from opendataqna import run_pipeline

final_sql, response, _resp = await run_pipeline(user_question,
                                                    user_database = 'imdb',
                                                    RUN_DEBUGGER=True,
                                                    EXECUTE_FINAL_SQL=True,
                                                    DEBUGGING_ROUNDS = 2, 
                                                    LLM_VALIDATION=True,
                                                    Embedder_model='vertex', # Options: 'vertex' or 'vertex-lang'
                                                    SQLBuilder_model= 'gemini-1.0-pro',
                                                    SQLChecker_model= 'gemini-1.0-pro',
                                                    SQLDebugger_model= 'gemini-1.0-pro',
                                                    Responder_model= 'gemini-1.0-pro',
                                                    num_table_matches = 5,
                                                    num_column_matches = 10,
                                                    table_similarity_threshold = 0.1,
                                                    column_similarity_threshold = 0.1, 
                                                    example_similarity_threshold = 0.1, 
                                                    num_sql_matches=3)

                                                    

print("*"*50 +"\nGenerated SQL\n"+"*"*50+"\n"+final_sql)
print("\n"+"*"*50 +"\nResults\n"+"*"*50)
display(response)
print("*"*50 +"\nNatural Response\n"+"*"*50+"\n"+_resp)



### 📊 **Create Charts for the results** (Run only when you have proper results in the above cells)
Agent provides two suggestive google charts to display on a UI with element IDs chart_div and chart_div_1

In [ ]:
from opendataqna import Visualize

chart_js=''
chart_js = Visualize.generate_charts(user_question,final_sql,response) #sending 
# print(chart_js["chart_div_1"])

In [ ]:
from IPython.display import HTML

html_code = f'''
<script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
<script type="text/javascript">
{chart_js["chart_div"]}
</script>
<div id="chart_div"></div>
'''

HTML(html_code)


In [ ]:
html_code = f'''
<script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
<script type="text/javascript">
{chart_js["chart_div_1"]}
</script>
<div id="chart_div_1"></div>
'''

HTML(html_code)

## 🗑 **Clean Up Notebook Resources**
Make sure to delete your Cloud SQL instance and BigQuery Datasets when your are finished with this notebook to avoid further costs. 💸 💰

Uncomment and run the cell below to delete 

In [ ]:
# # delete Cloud SQL instance
# !gcloud sql instances delete {PG_INSTANCE} -q

# #delete BigQuery Dataset using bq utility
# !bq rm -r -f -d {BQ_DATASET_NAME}

# #delete BigQuery 'Open Data QnA' Vector Store Dataset using bq utility

# !bq rm -r -f -d {BQ_OPENDATAQNA_DATASET_NAME}

